

Anomaly detection using Graph Convolutional Network
====================================


Model Overview
------------------------------------------
GCN from the perspective of message passing
```````````````````````````````````````````````
We describe a layer of graph convolutional neural network from a message
passing perspective; the math can be found `here <math_>`_.
It boils down to the following step, for each node $u$:

1) Aggregate neighbors' representations $h_{v}$ to produce an
intermediate representation $\hat{h}_u$.  2) Transform the aggregated
representation $\hat{h}_{u}$ with a linear projection followed by a
non-linearity: $h_{u} = f(W_{u} \hat{h}_u)$.

We will implement step 1 with DGL message passing, and step 2 with the
``apply_nodes`` method, whose node UDF will be a PyTorch ``nn.Module``.

GCN implementation with DGL
``````````````````````````````````````````
We first define the message and reduce function as usual.  Since the
aggregation on a node $u$ only involves summing over the neighbors'
representations $h_v$, we can simply use builtin functions:



In [1]:
%matplotlib inline

In [2]:
import random
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

NODES = 1000
EDGES = 20000


gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

We then define the node UDF for ``apply_nodes``, which is a fully-connected layer:



In [3]:
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

We then proceed to define the GCN module. A GCN layer essentially performs
message passing on all the nodes then applies the `NodeApplyModule`. Note
that we omitted the dropout in the paper for simplicity.



In [4]:
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(gcn_msg, gcn_reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

The forward function is essentially the same as any other commonly seen NNs
model in PyTorch.  We can initialize GCN like any ``nn.Module``. For example,
let's define a simple neural network consisting of two GCN layers. Suppose we
are training the classifier for the cora dataset (the input feature size is
1433 and the number of classes is 7).



In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCN(NODES, 16, F.relu)
        self.gcn2 = GCN(16, 2, F.relu)
    
    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.gcn2(g, x)
        return x
net = Net()
print(net)

Net(
  (gcn1): GCN(
    (apply_mod): NodeApplyModule(
      (linear): Linear(in_features=1000, out_features=16, bias=True)
    )
  )
  (gcn2): GCN(
    (apply_mod): NodeApplyModule(
      (linear): Linear(in_features=16, out_features=2, bias=True)
    )
  )
)


We load the cora dataset using DGL's built-in data module.



In [6]:
from dgl.data import citation_graph as citegrh

def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    print(data.labels)
    mask = th.ByteTensor(data.train_mask)
    g = data.graph
    # add self loop
    g.remove_edges_from(g.selfloop_edges())
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    return g, features, labels, mask

In [7]:
def load_my_data():
    g = dgl.DGLGraph()
    
    g.add_nodes(NODES)
    
    edge_list = []
    
    # create at least one edge from every node
    for node in range(g.number_of_nodes()):
        random_node = random.randint(0, g.number_of_nodes()-1)
        while random_node == node:
            random_node = random.randint(0, g.number_of_nodes()-1)
        edge_list.append((node, random_node))
    
    #print("Edge list:", edge_list)
    
    # create additional edges
    for _ in range(EDGES):
        random_node1 = random.randint(0, g.number_of_nodes()-1)
        random_node2 = random.randint(0, g.number_of_nodes()-1)
        
        while random_node1 == random_node2:
            random_node1 = random.randint(0, g.number_of_nodes()-1)
        edge = (random_node1, random_node2)
        r_edge = (random_node2, random_node1)
        
        if edge in edge_list or r_edge in edge_list:
            continue
        else:
            edge_list.append(edge)    
    
    #print("Edge list:", edge_list)
    # add edges two lists of nodes: src and dst
    src, dst = tuple(zip(*edge_list))
    g.add_edges(src, dst)
    # edges are directional in DGL; make them bi-directional
    g.add_edges(dst, src)
    return g

In [14]:
import networkx as nx
from avd.configs import config
from avd.datasets.twitter import load_data

def load_twitter(twitter_graph, twitter_config):
    g = dgl.DGLGraph()
    
    labels = {"neg": "Real", "pos": "Fake"}
    twitter_graph, twitter_config = load_data(dataset_file_name="twitter_filtered.csv", 
                                          labels_file_name="twitter_labels_filtered.csv", 
                                          labels_map=labels, 
                                          limit=5000000) # Loads filtered dataset.
    print(len(twitter_graph.vertices))
    
    """
    # look into graph_factory.py
    # Add nodes
    g.add_nodes(NODES)
    
    # Add edges
    edge_list = []
    for every line in edge file:
        src, dst = tuple(zip(*edge_list))
        g.add_edges(src, dst)
        
    # Add labels
    # Look in abstract_graph.py/write_nodes_labels()
    """
    #g.from_networkx(twitter_graph)#, edge_attrs=['e1', 'e2'])

    #return g, features, labels, mask
    return g

g = load_twitter(twitter_graph, twitter_config)

Loading labels...
Loading graph...
Data loaded.
2320975


We then train the network as follows:



In [ ]:
g = load_my_data()

features = th.eye(NODES)
labels_list = [0] * NODES
for i in range(NODES//4):
    node = random.randint(0, NODES-1)
    labels_list[node] = 1
labels = th.LongTensor(labels_list)

mask_list = [0] * NODES
for i in range(NODES//5):
    labels_list[i] = 1

mask = th.BoolTensor(mask_list)

print("Number of nodes:", g.number_of_nodes())
print("Number of edges:", g.number_of_edges())
print("Features:", type(features), features.shape)
print("Labels:", type(labels), labels.shape)
print("mask:", type(mask), mask.shape)

In [ ]:
import time
import numpy as np
#g, features, labels, mask = load_cora_data()
#print("Features:", type(features), features.shape)
#print("Labels:", type(labels), labels.shape)
#print("mask:", type(mask), mask.shape)

def acc(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
dur = []
for epoch in range(200):
    if epoch >=3:
        t0 = time.time()
        
    logits = net(g, features)
    #print("logits:", type(logits), logits.shape)
    #print("logits:", logits)
    logp = F.log_softmax(logits, 1)
    #print("logp:", type(logp), logp.shape)
    #print("logp[mask]:", logp[mask].shape)
    #print("mask.shape:", mask.shape, "logp.shape:", logp.shape, "logp[mask].shape:", logp[mask].shape, "labels.shape:",labels[mask].shape)
    
    loss = F.nll_loss(logp, labels)
    accuracy = acc(logits, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        dur.append(time.time() - t0)
    
    print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} | Time(s) {:.4f}".format(
            epoch+1, loss.item(), accuracy, np.mean(dur)))


GCN in one formula
------------------
Mathematically, the GCN model follows this formula:

$H^{(l+1)} = \sigma(\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}H^{(l)}W^{(l)})$

Here, $H^{(l)}$ denotes the $l^{th}$ layer in the network,
$\sigma$ is the non-linearity, and $W$ is the weight matrix for
this layer. $D$ and $A$, as commonly seen, represent degree
matrix and adjacency matrix, respectively. The ~ is a renormalization trick
in which we add a self-connection to each node of the graph, and build the
corresponding degree and adjacency matrix.  The shape of the input
$H^{(0)}$ is $N \times D$, where $N$ is the number of nodes
and $D$ is the number of input features. We can chain up multiple
layers as such to produce a node-level representation output with shape
:math`N \times F`, where $F$ is the dimension of the output node
feature vector.

The equation can be efficiently implemented using sparse matrix
multiplication kernels (such as Kipf's
`pygcn <https://github.com/tkipf/pygcn>`_ code). The above DGL implementation
in fact has already used this trick due to the use of builtin functions. To
understand what is under the hood, please read our tutorial on :doc:`PageRank <../../basics/3_pagerank>`.

